In [1]:
import pandas as pd
import numpy as np
import json
from pyearth import Earth
import plotly.graph_objects as go
import datetime as dt
from re import findall
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
import copy

In [2]:
def adjust_dt(df, delta_time):
    date_init =  df['snapped_at'][0]
    idx_date = df['snapped_at'].map(dt.datetime.toordinal)
    idx_date = idx_date - idx_date[0]
    df_modify = df[df['snapped_at'] == date_init]
    for i in idx_date:
        if i!=0:
            if i%delta_time == 0:
                df_modify = pd.concat([df_modify,df[df['snapped_at'] == date_init+pd.Timedelta(i,'d')]])
    return df_modify

In [3]:
f = open('forecast.json')
data_forecast = json.load(f)

df_forecast = pd.DataFrame({
    'coin' : data_forecast['coin'],
    'timeframe' : data_forecast['timeframe'],
    'date_before' : pd.to_datetime([d['date_before'] for d in data_forecast['detail']], format='%Y-%m-%d %H:%M:%S'),
    'date_after' : pd.to_datetime([d['date_after'] for d in data_forecast['detail']], format='%Y-%m-%d %H:%M:%S'),
    'fake_pump' : pd.to_datetime([d['fake_pump'] for d in data_forecast['detail']], format='%Y-%m-%d %H:%M:%S'),
})

In [4]:
df_forecast

,coin,timeframe,date_before,date_after,fake_pump
0,adp,1,2021-11-03,2022-01-02,2022-01-11
1,adx,3,2018-12-01,2019-10-03,2019-10-06
2,ae,3,2019-03-27,2019-04-26,2019-06-25
3,aion,5,2018-03-27,2019-08-19,2021-02-19
4,amp,1,2022-01-27,2022-03-13,2022-05-12
...,...,...,...,...,...
238,zec,6,2017-11-06,2020-03-07,2020-03-13
239,zec,3,2018-04-02,2018-06-16,2020-01-16
240,zen,6,2017-10-12,2020-05-17,2021-05-24
241,zen,5,2018-03-26,2018-05-15,2021-09-16


In [5]:
f = open('df_checked_dict.json')
data = json.load(f)

df_checked = {}
for i in data.keys():
    df_checked[i] = pd.DataFrame.from_dict(data[i])
    df_checked[i]['snapped_at'] = pd.to_datetime(df_checked[i]['snapped_at'], format='%Y-%m-%d %H:%M:%S UTC')

In [6]:
TIME_LENGTH = 60

arr_base_price = []
arr_max_price = []
arr_pct_chg = []
for i in range(len(df_forecast)):
    coin = df_forecast['coin'][i]
    temp_df = df_checked[coin]
    initial_date = df_forecast['fake_pump'][i]
    final_date = df_forecast['fake_pump'][i] + pd.Timedelta(TIME_LENGTH*df_forecast['timeframe'][i],'d')
    base_price_bound = df_forecast['fake_pump'][i] + pd.Timedelta(TIME_LENGTH/2*df_forecast['timeframe'][i],'d')
    temp_df2 = temp_df[temp_df['snapped_at'] > initial_date]
    temp_df2 = temp_df2[temp_df2['snapped_at'] < base_price_bound]
    base_price = np.percentile(temp_df2['price'],1)
    temp_df2 = temp_df[temp_df['snapped_at'] > initial_date]
    temp_df2 = temp_df2[temp_df2['snapped_at'] < final_date]
    max_price = max(temp_df2['price'])
    pct_chg = (max_price - base_price)/base_price
    arr_base_price.append(base_price)
    arr_max_price.append(max_price)
    arr_pct_chg.append(pct_chg)

df_forecast2 = copy.copy(df_forecast)
df_forecast2['base_price'] = arr_base_price
df_forecast2['max_price'] = arr_max_price
df_forecast2['pct_chg'] = arr_pct_chg

In [7]:
df_forecast2.sort_values(by=['pct_chg'])

,coin,timeframe,date_before,date_after,fake_pump,base_price,max_price,pct_chg
42,cusdc,1,2019-10-01,2020-04-27,2020-10-18,0.021094,0.021326,0.010967
205,usdt,1,2017-07-20,2017-12-17,2018-05-04,0.992288,1.006082,0.013902
177,sprt,1,2022-05-10,2022-06-20,2022-06-28,0.305027,0.309672,0.015228
160,rev,1,2020-11-16,2021-01-15,2021-01-19,0.011594,0.012544,0.081907
166,sb,1,2022-01-26,2022-02-17,2022-05-15,216.285836,236.371311,0.092865
...,...,...,...,...,...,...,...,...
148,pyr,2,2021-07-25,2021-09-17,2021-09-19,4.054318,44.144746,9.888327
43,cvc,5,2018-04-11,2019-09-03,2020-11-06,0.059539,0.718131,11.061457
135,omg,3,2019-10-22,2020-03-11,2020-03-14,0.462227,6.399716,12.845388
127,ngc,2,2021-04-28,2021-06-13,2021-09-19,0.080785,2.074769,24.682670


In [8]:
fig = go.Figure(data=[go.Histogram(x=df_forecast2['pct_chg'])])
fig.show()

In [9]:
fig = go.Figure()
fig.add_trace(go.Box(y=df_forecast2['pct_chg']))


fig.show()

In [12]:
print('probability return < 5%: ', len(df_forecast2['pct_chg'][df_forecast2['pct_chg'] <= 0.05])/len(df_forecast2['pct_chg']))
print('probability coin up to > 50%: ', len(df_forecast2['pct_chg'][df_forecast2['pct_chg'] >= 0.5])/len(df_forecast2['pct_chg']))
print('probability coin up to > 100%: ', len(df_forecast2['pct_chg'][df_forecast2['pct_chg'] >= 1])/len(df_forecast2['pct_chg']))
print('probability coin up to > 200%: ', len(df_forecast2['pct_chg'][df_forecast2['pct_chg'] >= 2])/len(df_forecast2['pct_chg']))
print('probability coin up to > 500%: ', len(df_forecast2['pct_chg'][df_forecast2['pct_chg'] >= 5])/len(df_forecast2['pct_chg']))

probability return < 5%:  0.012345679012345678
probability coin up to > 50%:  0.7654320987654321
probability coin up to > 100%:  0.4609053497942387
probability coin up to > 200%:  0.20987654320987653
probability coin up to > 500%:  0.06995884773662552


In [13]:
# averagenya --
# tolerance bottom
# visualisasi
# definisi bottom
np.average(df_forecast2['pct_chg'])

2.0364879230556587

In [14]:
np.median(df_forecast2['pct_chg'])

0.949480444017135